In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt

from shapely.ops import nearest_points
from shapely.geometry import Point

In [ ]:
gdf = gpd.read_file('rockoutcrop/add_rockoutcrop_landsatWGS84.shp').to_crs('epsg:3031')
df = pd.read_csv('DEM/bamber.5km97.dat', sep=' ', header=None, names=['latitude','longitude','difference','elevation'])

In [ ]:
df.loc[:,'latitude'] = np.around(df.loc[:,'latitude'],0)
df.loc[:,'longitude'] = np.around(df.loc[:,'longitude'],0)
meanElev = df.groupby(['latitude', 'longitude']).elevation.mean().reset_index()

In [ ]:
%%time
def find_nearest_elev(lon, lat):
    try:    return meanElev[(meanElev.longitude == int(lon)) & (meanElev.latitude == int(lat))].elevation.values[0]
    except: return None

gdf['elev'] = gdf.to_crs('epsg:4326').apply(lambda row: find_nearest_elev(row.geometry.centroid.x, row.geometry.centroid.y), axis=1)

In [ ]:
#Cropout
m = Basemap(projection='spstere',boundinglat=-60,lon_0=180,resolution='h')
xpt, ypt = m(gdf.to_crs('epsg:4326').geometry.centroid.x,gdf.to_crs('epsg:4326').geometry.centroid.y)

#Stazioni
stazioni = pd.read_csv('stazioni.csv', encoding= 'utf-8')
stazioni = gpd.GeoDataFrame(
    stazioni, geometry=gpd.points_from_xy(stazioni.longitude, stazioni.latitude))
stazioni = stazioni.set_crs('epsg:4326').to_crs('epsg:3031')

sxpt, sypt = m(stazioni.to_crs('epsg:4326').geometry.centroid.x, stazioni.to_crs('epsg:4326').geometry.centroid.y)

In [ ]:
plt.figure(figsize=(15,15))
m.drawcoastlines()
m.fillcontinents(color='white',lake_color='aqua')
m.drawmapboundary(fill_color='lightblue')

m.scatter(xpt, ypt, c=np.log(gdf['geometry'].area), cmap=plt.cm.bwr, zorder = 3, s=0.1)
plt.colorbar(pad = 0.01 , shrink = 0.7, aspect = 50)
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
m = Basemap(projection='spstere',boundinglat=-60,lon_0=180,resolution='h')
m.drawcoastlines()
m.fillcontinents(color='white',lake_color='aqua')
m.drawmapboundary(fill_color='lightblue')

m.scatter(xpt, ypt, c=gdf['elev'], cmap=plt.cm.bwr, zorder = 3, s=0.1)
m.scatter(sxpt, sypt, c='black', zorder = 3, s=50)
plt.colorbar(pad = 0.01 , shrink = 0.7, aspect = 50)
plt.show()

##### EXTRACT ROCKs or UNCONSOLIDATED AREAs

In [ ]:
%%time
gudf = gpd.read_file('GeoUnits/shapefile/geo_units.shp')

unconsolidatedClasses = ['Hs', 'Qk', 'Quc','Qc','Qu','Qa','Qs']
topographyClasses = ['w', 'water', 'ice', '?', 'unknown']

unconsolidated = gudf[gudf.MAPSYMBOL.isin(unconsolidatedClasses)]
rocks = gudf[~(gudf.MAPSYMBOL.isin(unconsolidatedClasses)) & ~(gudf.MAPSYMBOL.isin(topographyClasses))]

del gudf

In [ ]:
#Cropout
rocks_x, rocks_y = m(rocks.geometry.centroid.x,rocks.geometry.centroid.y)
unco_x, unco_y   = m(unconsolidated.geometry.centroid.x,unconsolidated.geometry.centroid.y)

In [ ]:
plt.figure(figsize=(15,15))
m = Basemap(projection='spstere',boundinglat=-60,lon_0=180,resolution='h')
m.drawcoastlines()
m.fillcontinents(color='white',lake_color='aqua')
m.drawmapboundary(fill_color='lightblue')

m.scatter(rocks_x, rocks_y, c='gray', zorder = 3, s=0.1)
m.scatter(unco_x, unco_y, c='yellow', zorder = 3, s=0.1)
plt.show()

In [ ]:
clusterDF = pd.DataFrame({'lon' : gdf.geometry.centroid.x, 
                          'lat' : gdf.geometry.centroid.y,
                          'area': gdf.geometry.area,
                          'elev': gdf.elev})
clusterDF = gpd.GeoDataFrame(
    clusterDF, geometry=gpd.points_from_xy(clusterDF.lon, clusterDF.lat)).set_crs('epsg:3031')

In [ ]:
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import numpy as np

X=clusterDF.iloc[:,:-1]
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)

In [ ]:
%%time
inertia = []
for i in range(1,11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,11),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
        dict(
            x=3,
            y=inertia[2],
            xref="x",
            yref="y",
            text="Elbow!",
            showarrow=True,
            arrowhead=7,
            ax=20,
            ay=-40
        )
    ])

In [ ]:
clusterDF

In [ ]:
#Cropout
clust_x, clust_y = m(clusterDF.to_crs('epsg:4326').geometry.x,clusterDF.to_crs('epsg:4326').geometry.y)

In [ ]:
from scipy.spatial import ConvexHull as ch
from shapely.geometry import Polygon
import shapely
from descartes import PolygonPatch
from matplotlib.collections import PatchCollection
import matplotlib

def appendCluster(clusters, point):
        for i in range(num_cluster):
            if clusters.geometry[i].contains(point):
                return i
        return -1

def getElevRange():
    elevRange = []
    for i in range(num_cluster):
        temp = gdf[gdf.cluster == i]
        elevRange.append(np.max(temp.elev) - np.min(temp.elev))
    return pd.Series(elevRange)

num_cluster = 1

def getClusters(n):
    global num_cluster
    num_cluster = n
    kmeans = KMeans(n_clusters=num_cluster, init="k-means++",n_init=10,tol=1e-04, random_state=42)
    kmeans.fit(X)
    
    m = Basemap(projection='spstere',boundinglat=-60,lon_0=180,resolution='h') 
    
    clusterDF['label']=kmeans.labels_
    clusters = pd.DataFrame(columns=('cluster_num','geometry'))    
    for label in range(0,num_cluster):
        arr = clusterDF[clusterDF.label == label][['lon','lat']].to_numpy()
        hull = ch(arr)
        polylist = []
        for idx in hull.vertices: #Indices of points forming the vertices of the convex hull.
            polylist.append(arr[idx]) #Append this index point to list
        p = Polygon(polylist)
        clusters.at[label] = [label,p]
    clusters = gpd.GeoDataFrame(clusters, crs='EPSG:{}'.format(3031), geometry='geometry')
    clusters['centroid'] = clusters.geometry.apply(lambda x: x.centroid)
    CCxpt, CCypt = m(clusters.centroid.to_crs('epsg:4326').x, clusters.centroid.to_crs('epsg:4326').y)
    gdf['cluster'] = gdf.geometry.apply(lambda x: appendCluster(clusters, x.centroid))
    
    patches = []
    for poly in clusters.to_crs('epsg:4326').geometry:
        if poly.geom_type == 'Polygon':
            mpoly = shapely.ops.transform(m, poly)
            patches.append(PolygonPatch(mpoly))
        elif poly.geom_type == 'MultiPolygon':
            for subpoly in poly:
                mpoly = shapely.ops.transform(m, poly)
                patches.append(PolygonPatch(mpoly))
        else:
            print(poly, 'is neither a polygon nor a multi-polygon. Skipping it')
            
    clusters['elev'] = gdf.groupby('cluster').elev.mean().reset_index().sort_values(by='cluster').iloc[1:].elev.values
    clusters['elevation_range'] = getElevRange()
    
    #clusters = clusters.to_crs('EPSG:4326')
    
    fig, ax1 = plt.subplots(figsize=(20, 20))
    m.drawcoastlines()
    m.fillcontinents(color='white',lake_color='aqua')
    m.drawmapboundary(fill_color='lightblue')
    m.scatter(clust_x, clust_y, c=kmeans.labels_, zorder = 3, s=0.1)
    p = PatchCollection(patches, cmap=matplotlib.cm.jet, match_original=True, zorder=4, alpha=0.5)
    p.set_array(clusters.elev)
    ax1.add_collection(p)
    plt.colorbar(p, pad = 0.01 , shrink = 0.85, aspect = 20)
    m.scatter(CCxpt, CCypt, marker='P', c='white', zorder = 4, s=100)
    m.scatter(sxpt, sypt, marker='^', c='black', zorder = 4, s=150)
    plt.show()
    return clusters

In [ ]:
clusters = getClusters(8)

In [ ]:
clusters

In [ ]:
coastline = gpd.read_file('coastline/add_coastline_medium_res_line_v7_4.shp')

In [ ]:
distance_from_coastline = []
for point in range(len(stazioni)):
    p1 = stazioni.iloc[point].geometry
    p2 = coastline.iloc[np.argmin([stazioni.iloc[point].geometry.distance(line) for line in coastline.geometry])].geometry

    ax = coastline.plot()
    ax.scatter(p1.x, p1.y, c='black', zorder=3)
    ax.scatter(p2.centroid.x, p2.centroid.y, c='red', zorder=3)
    distance = p1.distance(p2) / 1000
    distance_from_coastline.append(distance)
    print(distance)
    plt.show()

In [ ]:
stazioni['distance_from_coastline(km)'] = np.around(distance_from_coastline,2)

In [ ]:
stazioni

In [ ]:
import pyproj    
import shapely
import shapely.ops as ops
from shapely.geometry.polygon import Polygon
from functools import partial


def getArea(poly):
    geom = poly
    geom_area = ops.transform(
        partial(
            pyproj.transform,
            pyproj.Proj(init='EPSG:4326'),
            pyproj.Proj(
                proj='aea',
                lat_1=geom.bounds[1],
                lat_2=geom.bounds[3]
            )
        ),
        geom)
    return round(geom_area.area / 1e6)

In [ ]:
clusters['area_km2'] = clusters.to_crs('epsg:4326').geometry.apply(lambda x: getArea(x))

In [ ]:
clusters.head()

In [ ]:
import plotly.express as px
fig = px.scatter(clusters, x="cluster_num", y="elev", size='area_km2', color='elevation_range',width=1000, height=500)
fig.show()

In [ ]:
%%time
def get_total_cropout_area_under_radius(point, target, radius):
    arr = [point.distance(geom) / 1000 for geom in target]
    idx = [idx for idx in range(len(arr)) if arr[idx] < radius]
    return np.around(target.iloc[idx].geometry.area.values.sum()/1e6,2)

stazioni['nearest_cluster']             = stazioni.geometry.apply(lambda x: clusters.iloc[np.argmin([x.distance(geom.centroid) / 1000 for geom in clusters.geometry])].cluster_num )
stazioni['distance_to_nearest_cluster'] = stazioni.geometry.apply(lambda x: np.around(x.distance(clusters.iloc[np.argmin([x.distance(geom.centroid) / 1000 for geom in clusters.geometry])].centroid) / 1000,2 ))
stazioni['nearest_cropout(km)']         = stazioni.geometry.apply(lambda x: np.around(np.min([x.distance(geom) / 1000 for geom in gdf.geometry]), 2) )
stazioni['cropout_area_within_500km']   = stazioni.geometry.apply(lambda x: get_total_cropout_area_under_radius(x, gdf.geometry, 500))
stazioni['cropout_area_within_1000km']  = stazioni.geometry.apply(lambda x: get_total_cropout_area_under_radius(x, gdf.geometry, 1000))

In [ ]:
stazioni

In [ ]:
fig = px.scatter(stazioni, x="distance_from_coastline(km)", y="distance_to_nearest_cluster", size='cropout_area_within_1000km', color='dust_conc_(ppb)',width=1000, height=500)
fig.show()

In [ ]:
fig = px.scatter(stazioni, x="dust_conc_(ppb)", y="cropout_area_within_1000km", size='distance_from_coastline(km)', color='altitude',width=1000, height=500)
fig.show()